# Concatenate boreholes

In [1]:
%matplotlib widget

In [2]:
config Completer.use_jedi = False

In [3]:
import datetime

In [4]:
import numpy as np

In [5]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

Lecture des fichiers de position des forages

In [6]:
v8 = gpd.read_file('../../CF_data/Data_UMONS/SIG/Pilote_v8.0.shp')
v8.query('GPS2021=="NON"')

,Id,Type,Pilote,Ref,diam,Valid,Syscal,Impo,Inox,Impo_perm,GPS2021,geometry
18,0,4,1,400,0,0,0,0,0,0,NON,POINT (152885.032 122585.367)
32,0,5,1,500,0,0,0,0,0,0,NON,POINT (152890.824 122583.580)
63,0,3,1,306,0,0,0,0,0,0,NON,POINT (152867.452 122588.877)


## Extraction des dates de forages du type de forage et du foreur de l'étude de caractérisation

In [7]:
# informations sur les forages de l'étude de caractérisation de 2010
Logs_forages = pd.read_excel('../../CF_data/Data_SITEREM/Logs forages.xls',sheet_name='Forages, tranchées')

In [8]:
Logs_forages = Logs_forages[['Date', 'N°', 'Id', 'Profondeur', 'Description', 'Piézo', 'Unnamed: 6',
                             'Gouge Ø75', 'MFT Ø145', 'carottier', 'tarrière', 'Liner Ø60']] 

In [9]:
Logs_forages.rename({'Gouge Ø75':'Gouge_75', 'Liner Ø60': 'Liner_60'}, axis=1, inplace=True)

In [23]:
Logs_forages.loc[Logs_forages.query('Profondeur!=Profondeur' ).index,'Profondeur']=''

In [31]:
forages = Logs_forages.query('Profondeur.str.contains("Forage")  and Profondeur!="Forage bloqué"', engine='python')

In [32]:
forages['Ref'] = forages['Profondeur'].apply(lambda x: x.split('Forage ')[1].replace(' ', ''))

<ipython-input-32-07818ba03c75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['Ref'] = forages['Profondeur'].apply(lambda x: x.split('Forage ')[1].replace(' ', ''))


In [33]:
forages.Ref

0           1
9           2
19          3
27          4
33          5
        ...  
1973     612c
1977      734
1984    FP595
1994      540
2004      600
Name: Ref, Length: 281, dtype: object

In [34]:
forages['tmp'] = forages['Piézo'].replace(np.nan, '', regex=True) + '-' + forages['Unnamed: 6'].replace(np.nan, '', regex=True) + '-' + forages['Gouge_75'].replace(np.nan, '', regex=True)

<ipython-input-34-0d7b933e2cc3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['tmp'] = forages['Piézo'].replace(np.nan, '', regex=True) + '-' + forages['Unnamed: 6'].replace(np.nan, '', regex=True) + '-' + forages['Gouge_75'].replace(np.nan, '', regex=True)


In [35]:
forages['Piezo'] = forages['tmp'].apply(lambda x: x.lstrip(' ').split('-')[0].lower()=='x') 

<ipython-input-35-fc78262c80a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['Piezo'] = forages['tmp'].apply(lambda x: x.lstrip(' ').split('-')[0].lower()=='x')


In [36]:
forages['Company'] = forages['tmp'].apply(lambda x: x.rstrip('-').split('-')[-1].upper()) 

<ipython-input-36-4da94ebfe204>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forages['Company'] = forages['tmp'].apply(lambda x: x.rstrip('-').split('-')[-1].upper())


In [37]:
forages.query('Company=="X"')

,Date,N°,Id,Profondeur,Description,Piézo,Unnamed: 6,Gouge_75,MFT Ø145,carottier,tarrière,Liner_60,Ref,tmp,Piezo,Company
123,2010-01-18,15.0,124.0,Forage FP15,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,FP15,X--,True,X


In [38]:
forages.loc[123, 'Company']='SITEREM'

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [41]:
forages = forages[['Date', 'Ref', 'Company', 'Piezo']]
forages

,Date,Ref,Company,Piezo
0,2010-03-11,1,SBS ENVIRONNEMENT,True
9,2010-03-11,2,SBS ENVIRONNEMENT,False
19,2010-03-11,3,SBS ENVIRONNEMENT,False
27,2010-03-10,4,SBS ENVIRONNEMENT,False
33,2010-03-12,5,SBS ENVIRONNEMENT,False
...,...,...,...,...
1973,2010-08-03,612c,SBS ENVIRONNEMENT,False
1977,2010-08-03,734,SITEREM,True
1984,2010-09-01,FP595,SITEREM,True
1994,2010-09-02,540,SITEREM,True


In [46]:
forages['Ref'] = forages['Date'].apply(lambda x : str(x.year) + '-') + forages['Ref']

In [47]:
forages

,Date,Ref,Company,Piezo
0,2010-03-11,2010-1,SBS ENVIRONNEMENT,True
9,2010-03-11,2010-2,SBS ENVIRONNEMENT,False
19,2010-03-11,2010-3,SBS ENVIRONNEMENT,False
27,2010-03-10,2010-4,SBS ENVIRONNEMENT,False
33,2010-03-12,2010-5,SBS ENVIRONNEMENT,False
...,...,...,...,...
1973,2010-08-03,2010-612c,SBS ENVIRONNEMENT,False
1977,2010-08-03,2010-734,SITEREM,True
1984,2010-09-01,2010-FP595,SITEREM,True
1994,2010-09-02,2010-540,SITEREM,True


## forages phase 1

In [ ]:
v2017 = gpd.read_file('../../CF_data/Data_UMONS/SIG/sondages_v2017-05-18.shp')
v2017

In [ ]:
v2017['Ref'] = v2017['Date'].apply(lambda x : str(datetime.datetime.strptime(x, '%Y-%m-%d').year) + '-') + v2017['Ref_sond']

In [ ]:
v2017

In [ ]:
sond_ant = pd.read_excel('../../CF_data/Data_UMONS/Logs_forages_vUmons_2018-03-20.xlsx', sheet_name='Sond_ant')

In [ ]:
sond_ant['Ref'] = sond_ant['Ref_siterem'].apply(lambda x: '2010-' + str(x))

In [ ]:
sond_ant

In [ ]:
sond_ant_gdf = gpd.GeoDataFrame(sond_ant, geometry=gpd.points_from_xy(sond_ant.X, sond_ant.Y, crs=str('EPSG:31370')))

Lecture de l'extension du projet MEMORIS

In [ ]:
ext = gpd.read_file('../../CF_data/Data_UMONS/SIG/Site_Memoris.shp')

Affichage carte

In [ ]:
def label(gdf, field, ax, size='xx-small'):
    for x, y, label in zip(gdf.geometry.x, gdf.geometry.y, gdf[field]):
        ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords='offset points', size=size)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ext.plot(ax=ax, color='grey')
v2017.plot(ax=ax, color='r')

#for x, y, label in zip(v2017.geometry.x, v2017.geometry.y, v2017.Ref_sond):
#    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
label(v2017, 'Ref', ax)
v8.plot(ax=ax, color='g')
label(v8, 'Ref', ax)  
sond_ant_gdf.plot(ax=ax, color='b', markersize=1)
label(sond_ant_gdf, 'Ref', ax)

En ignorant Sond_ant_gdf

In [ ]:
v8['new_Ref'] = 'F'+v8['Ref']

In [ ]:
BH = []
BH.extend(v2017['Ref_sond'].to_list())
BH.extend(v8['new_Ref'].to_list())

In [ ]:
v8['ID'] = v8.new_Ref
v2017['ID'] = v2017.Ref_sond

In [ ]:
v8['Z']=101.

In [ ]:
gdf = gpd.GeoDataFrame( pd.concat( [v8[['geometry','ID', 'Z']], v2017[['geometry','ID', 'Z']]], ignore_index=True) )

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ext.plot(ax=ax, color='lightgrey')
gdf.plot(ax=ax, markersize=1)
label(gdf, 'ID', ax)

In [ ]:
gdf['X']=gdf.geometry.x
gdf['Y']=gdf.geometry.y

In [ ]:
pd.DataFrame(gdf)[['ID', 'X', 'Y']].to_csv('../../CF_data/synthese/boreholes.csv', index=False)

Liste de tous les ouvrages références y compris les doublons entre jeux de données (et dans chaque jeu de donnée) 

In [ ]:
BH = []
BH.extend(v2017['Ref_sond'].to_list())
BH.extend(v8['Ref'].to_list())
BH.extend(sond_ant_gdf['Ref_siterem'].to_list())

In [ ]:
gdf = v2017.merge(sond_ant_gdf, how='outer', left_on='Ref_sond', right_on='Ref_siterem')
gdf[['Ref_sond', 'Ref_siterem']][0:100]

In [ ]:
gdf.query('Ref_sond==Ref_siterem')['Ref_sond']

In [ ]:
gdf.query('Ref_sond!=Ref_sond')['Ref_siterem'].to_list()

In [ ]:
len(BH)

In [ ]:
len(set(BH))

In [ ]:
gdf.query('Ref_siterem!=Ref_siterem')['Ref_sond'].to_list()

In [ ]:
gdf.columns

In [ ]:
for idx, row in gdf.iterrows():
    if row['geometry_x'] is None:
        if row['geometry_y'] is not None:
            gdf.loc[idx, 'distance'] = 0.
        else:
            gdf.loc[idx, 'distance'] = None
    else:
        if row['geometry_y'] is None:
            gdf.loc[idx, 'distance'] = 0.
        else:
            gdf.loc[idx, 'distance'] = row['geometry_x'].distance(row['geometry_y'])

In [ ]:
gdf.query('distance != distance') # recherche de forages n'ayant aucune coordonnées

In [ ]:
gdf.query('distance>0.')